In [341]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [337]:
ngl_base = 'https://neuromancer-seung-import.appspot.com/' # this will point to the public bucket


In [798]:
def make_json(name, seg_ids):
    state = {
        "layers": [
            {
            "source": "graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod",
            "type": "segmentation_with_graph",
            "selectedAlpha": 0.3,
            "segments": seg_ids,
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "graphOperationMarker": [
                {
                "annotations": [],
                "tags": []
                },
                {
                "annotations": [],
                "tags": []
                }
            ],
            "pathFinder": {
                "color": "#ffff00",
                "pathObject": {
                "annotationPath": {
                    "annotations": [],
                    "tags": []
                },
                "hasPath": False
                }
            },
            "name": name
            },
            {
            "type": "segmentation",
            "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
            "objectAlpha": 0.1,
            "hideSegmentZero": False,
            "ignoreSegmentInteractions": True,
            "segmentColors": {
                "1": "#bfbfbf",
                "2": "#d343d6"
            },
            "segments": [
                "1",
                "2"
            ],
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "name": "neuropil"
            }
        ],
        "navigation": {
            "pose": {
            "position": {
                "voxelSize": [
                4.300000190734863,
                4.300000190734863,
                45
                ],
                "voxelCoordinates": [
                38930,
                138652,
                388
                ]
            }
            },
            "zoomFactor": 2
        },
        "showAxisLines": False,
        "showDefaultAnnotations": False,
        "perspectiveOrientation": [
            0,
            1,
            0,
            0
        ],
        "perspectiveZoom": 7704.152500126457,
        "showSlices": False,
        "selectedLayer": {
            "layer": "neuropil"
        },
        "layout": {
            "type": "3d",
            "orthographicProjection": True
        }
        }

    return state

In [389]:
# import dataframe
wing_df = pd.read_pickle('preMN_to_MN_wing_v604.pkl')

In [799]:
# generate a link based on a cell type
# only for local and intersegmental preMNs

idx = pd.IndexSlice
df = wing_df.loc[idx[:,:,:,['local','local_intersegmental'],:]].reset_index() # group by cell type

cell_types = df.cell_type.unique()

for cell_type in cell_types[9:10]:
    cell_type_label = str(cell_type)
    seg_ids = df[df.cell_type.isin([cell_type_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    # sb = StateBuilder(layers = [seg_layer,vol_layer_config], resolution=[4.3,4.3,45] , view_kws=view_options)
    


In [800]:
state = make_json(name = cell_type_label,
                   seg_ids = formatted_ids)

In [801]:
state

{'layers': [{'source': 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod',
   'type': 'segmentation_with_graph',
   'selectedAlpha': 0.3,
   'segments': ['648518346475198520',
    '648518346496932524',
    '648518346486488241',
    '648518346482660393',
    '648518346484500995',
    '648518346494164871',
    '648518346509933862',
    '648518346511783024',
    '648518346480341307',
    '648518346487932240',
    '648518346477534222',
    '648518346514170078',
    '648518346486727577',
    '648518346492596411',
    '648518346488766477',
    '648518346486175586',
    '648518346478681684',
    '648518346491367354',
    '648518346480946176'],
   'skeletonRendering': {'mode2d': 'lines_and_points', 'mode3d': 'lines'},
   'graphOperationMarker': [{'annotations': [], 'tags': []},
    {'annotations': [], 'tags': []}],
   'pathFinder': {'color': '#ffff00',
    'pathObject': {'annotationPath': {'annotations': [], 'tags': []},
     'hasPath': False}},
   'name': '2A'},
  {'type':

In [802]:
new_id = client.state.upload_state_json(state)
new_id

5986275473489920

In [803]:
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5986275473489920'